The goal of this project is connect to a website called "Buscalibre" which is an only library and pull in the data related with an specific search of books, categories, etc.
Then clean and create a DataFrame to save the data such as the book name, author, price, edition etc, into a csv file.


In [1]:
# Import libraries

import pandas as pd
from bs4 import BeautifulSoup
import requests
import datetime
import csv

In [2]:
# Connect to website and pull in and cleaning the data 
# Create the lists to save the data
id_tags= []
links = []
Titles = []
Authors = []
Num_Opinions = []
Info = []
Units_Left = []
Discount = []
Original_Price = []
Actual_Price = []
# The url after the searching the 'siddartha' book
url = 'https://www.buscalibre.com.co/libros/search?q=siddartha'
response = requests.get(url = url)
soup = BeautifulSoup(response.content,"html.parser")
soup1 = BeautifulSoup(soup.prettify(),"html.parser")

# It is possible that one search throw a few or hundred and thousand of results (books), and the website does not show all the results (if they are many) in one page, so the condition below
# ensure when is necesary to pull the data from multiple pages in the website or just one page.

if (soup1.find(name='a',id='pagnNextLink') != None):
    while(True):
        try:
            
            response = requests.get(url = url)
            soup = BeautifulSoup(response.content,"html.parser")
            soup1 = BeautifulSoup(soup.prettify(),"html.parser")

            next_url = soup1.find(name='a',id='pagnNextLink').get('href')
            url = next_url
            all_Products = soup1.find(name='div', class_='productos pais46')
            all_Tags = all_Products.find_all('div')

            for tag in all_Tags:
                try:
                    
                    id_tags.append(tag['data-id_producto'])
                    links.append(tag.find('a').get('href'))
                    Titles.append(tag.find(name='h3',class_='nombre margin-top-10 text-align-left').get_text().strip())
                    Authors.append(tag.find(name='div',class_='autor').get_text().strip())
                    Num_Opinions.append(tag.find(name='div',class_='box-stars hide-on-hover').get_text().strip())
                    Info.append(tag.find(name='div',class_='autor color-dark-gray metas hide-on-hover').get_text().strip())
                    Units_Left.append(tag.find(name='div',class_='stock hide-on-hover color-green font-size-small margin-top-5').get_text().strip())
                    Discount.append(tag.find(name='div',class_='descuento-v2 color-white position-relative').get_text().strip())
                    Original_Price.append(tag.find(name='p',class_='precio-antes hide-on-hover margin-0 color-dark-gray font-weight-normal').get_text().strip())
                    if tag.find(name='p',class_='precio-ahora hide-on-hover margin-0 font-size-medium') != None:
                        Actual_Price.append(tag.find(name='p',class_='precio-ahora hide-on-hover margin-0 font-size-medium').get_text().strip())
                    else: 
                        Actual_Price.append('Sold Out')
                    
                except KeyError:
                    continue
        except:
            break
else: 
    all_Products = soup1.find(name='div', class_='productos pais46')   
    all_Tags = all_Products.find_all('div')

    for tag in all_Tags:
        try:
            id_tags.append(tag['data-id_producto'])
            links.append(tag.find('a').get('href'))
            Titles.append(tag.find(name='h3',class_='nombre margin-top-10 text-align-left').get_text().strip())
            Authors.append(tag.find(name='div',class_='autor').get_text().strip())
            Num_Opinions.append(tag.find(name='div',class_='box-stars hide-on-hover').get_text().strip())
            Info.append(tag.find(name='div',class_='autor color-dark-gray metas hide-on-hover').get_text().strip())
            Units_Left.append(tag.find(name='div',class_='stock hide-on-hover color-green font-size-small margin-top-5').get_text().strip())
            Discount.append(tag.find(name='div',class_='descuento-v2 color-white position-relative').get_text().strip())
            Original_Price.append(tag.find(name='p',class_='precio-antes hide-on-hover margin-0 color-dark-gray font-weight-normal').get_text().strip())
            if tag.find(name='p',class_='precio-ahora hide-on-hover margin-0 font-size-medium') != None:
                Actual_Price.append(tag.find(name='p',class_='precio-ahora hide-on-hover margin-0 font-size-medium').get_text().strip())
            else: 
                Actual_Price.append('Sold Out')
        except KeyError:
                    continue
                    

In [3]:
# Clean up the data a little bit

Clean_Discount = []
for j in range(0,len(Discount)):
    Discount[j] = Discount[j].split('%')
    Discount[j][-1] = Discount[j][-1].strip()
    Clean_Discount.append(Discount[j][0])

    Info[j] =Info[j].split(',')
    Info[j][-1] = Info[j][-1].strip()

    Original_Price[j] = Original_Price[j].replace('$','')
    Actual_Price[j] = Actual_Price[j].replace('$','') 

    if Num_Opinions[j] == '':
        Num_Opinions[j] = '(0)'
        


In [4]:
# Create the DataFrame with all the data 

Nombre_Columnas = ['Title','Author','Original Price (COP)','Actual Price (COP)','Discount %','Units Left','Link','Number of Opinions','Extra Information']
df = pd.DataFrame(list(zip(Titles,Authors,Original_Price,Actual_Price,Clean_Discount,Units_Left,links,Num_Opinions,Info)),columns=Nombre_Columnas)
df['Date'] = datetime.date.today()
df

,Title,Author,Original Price (COP),Actual Price (COP),Discount %,Units Left,Link,Number of Opinions,Extra Information,Date
0,Siddartha,Hesen,28.000,21.000,25,Quedan 89 unidades,https://www.buscalibre.com.co/libro-siddartha/...,(0),"[, Nuevo]",2023-10-21
1,Siddartha,Array,101.069,50.534,50,Quedan 100+ unidades,https://www.buscalibre.com.co/libro-siddartha/...,(0),"[Createspace Independent Publishing Platform, ...",2023-10-21
2,Siddhartha,Hermann Hesse,129.700,64.850,50,Quedan 5 unidades,https://www.buscalibre.com.co/libro-siddhartha...,(14),"[Debolsillo, 2020, 1ª edición, Tapa dura, N...",2023-10-21
3,Siddhartha (Ed. Escolar),Hermann Hesse,46.000,27.600,40,Quedan 69 unidades,https://www.buscalibre.com.co/libro-siddhartha...,(12),"[Debolsillo, 2010, Tapa blanda, Nuevo]",2023-10-21
4,Siddhartha (tapa dura),Hermann Hesse,49.900,29.940,40,Quedan 2 unidades,https://www.buscalibre.com.co/libro-siddhartha...,(1),"[Lucemar, 2020, Tapa Dura, Nuevo]",2023-10-21
...,...,...,...,...,...,...,...,...,...,...
267,Select Case Studies on Accounting Scandals: Se...,Siddhartha Sankar Saha,590.596,295.298,50,Quedan 50 unidades,https://www.buscalibre.com.co/libro-select-cas...,(0),"[LAP LAMBERT Academic Publishing, Tapa Blanda...",2023-10-21
268,Regulating Cell to Cell Gap,Siddhartha Mitra,617.584,308.792,50,Quedan 50 unidades,https://www.buscalibre.com.co/libro-regulating...,(0),"[Lap Lambert Acad. Publ., Tapa blanda, Nuevo]",2023-10-21
269,Statutory Auditors' Objectivity in Corporate A...,Siddhartha Sankar Saha,557.825,278.913,50,Quedan 50 unidades,https://www.buscalibre.com.co/libro-statutory-...,(0),"[Lap Lambert Academic Publishing, Tapa blanda...",2023-10-21
270,Confluence of Artificial Intelligence and Robo...,"Bhattacharyya Siddhartha Bhattacharyya (), Ban...",394.232,197.116,50,Quedan 3 unidades,https://www.buscalibre.com.co/libro-confluence...,(0),"[Springer Nature B.V. 2023-03-14, , Tapa Blan...",2023-10-21


In [5]:
# DataFrame without the Sold Out books

index_toDrop = df[df['Actual Price (COP)']=='Sold Out'].index
df_copy = df.drop(index_toDrop)
df_copy

,Title,Author,Original Price (COP),Actual Price (COP),Discount %,Units Left,Link,Number of Opinions,Extra Information,Date
0,Siddartha,Hesen,28.000,21.000,25,Quedan 89 unidades,https://www.buscalibre.com.co/libro-siddartha/...,(0),"[, Nuevo]",2023-10-21
1,Siddartha,Array,101.069,50.534,50,Quedan 100+ unidades,https://www.buscalibre.com.co/libro-siddartha/...,(0),"[Createspace Independent Publishing Platform, ...",2023-10-21
2,Siddhartha,Hermann Hesse,129.700,64.850,50,Quedan 5 unidades,https://www.buscalibre.com.co/libro-siddhartha...,(14),"[Debolsillo, 2020, 1ª edición, Tapa dura, N...",2023-10-21
3,Siddhartha (Ed. Escolar),Hermann Hesse,46.000,27.600,40,Quedan 69 unidades,https://www.buscalibre.com.co/libro-siddhartha...,(12),"[Debolsillo, 2010, Tapa blanda, Nuevo]",2023-10-21
4,Siddhartha (tapa dura),Hermann Hesse,49.900,29.940,40,Quedan 2 unidades,https://www.buscalibre.com.co/libro-siddhartha...,(1),"[Lucemar, 2020, Tapa Dura, Nuevo]",2023-10-21
...,...,...,...,...,...,...,...,...,...,...
267,Select Case Studies on Accounting Scandals: Se...,Siddhartha Sankar Saha,590.596,295.298,50,Quedan 50 unidades,https://www.buscalibre.com.co/libro-select-cas...,(0),"[LAP LAMBERT Academic Publishing, Tapa Blanda...",2023-10-21
268,Regulating Cell to Cell Gap,Siddhartha Mitra,617.584,308.792,50,Quedan 50 unidades,https://www.buscalibre.com.co/libro-regulating...,(0),"[Lap Lambert Acad. Publ., Tapa blanda, Nuevo]",2023-10-21
269,Statutory Auditors' Objectivity in Corporate A...,Siddhartha Sankar Saha,557.825,278.913,50,Quedan 50 unidades,https://www.buscalibre.com.co/libro-statutory-...,(0),"[Lap Lambert Academic Publishing, Tapa blanda...",2023-10-21
270,Confluence of Artificial Intelligence and Robo...,"Bhattacharyya Siddhartha Bhattacharyya (), Ban...",394.232,197.116,50,Quedan 3 unidades,https://www.buscalibre.com.co/libro-confluence...,(0),"[Springer Nature B.V. 2023-03-14, , Tapa Blan...",2023-10-21


In [6]:
df.to_csv('/Users/Janus/Documents/GitHub/PortfolioProjects/WebScraping_Project/BuscaLibreScrapperDataSet.csv',index=False)